In [2]:
# train_mlp.py
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.callbacks import CSVLogger
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import sys, os

# --- Cấu hình ---
INPUT_FILE = 'npm_shuffled.csv'

def train_mlp_model(input_file):
    print(f"--- Huấn luyện mô hình Multi-Layer Perceptron (MLP) ---")
    
    # Bước 1: Tải và chuẩn bị dữ liệu
    try:
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{input_file}'.")
        return

    X = df.drop(columns=['label', 'package_name'])
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    print(f"[1/3] Đã chuẩn bị xong dữ liệu.")

    # Bước 2: Xây dựng và Huấn luyện mô hình MLP
    print("[2/3] Xây dựng và huấn luyện mô hình MLP...")
    model = tf.keras.models.Sequential([
        # Lớp đầu vào với số nơ-ron bằng số đặc tính
        tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
        # Lớp ẩn đầu tiên với 64 nơ-ron và hàm kích hoạt ReLU
        tf.keras.layers.Dense(64, activation='relu'),
        # Lớp Dropout để chống overfitting
        tf.keras.layers.Dropout(0.3),
        # Lớp ẩn thứ hai
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        # Lớp đầu ra với 1 nơ-ron và hàm sigmoid cho phân loại nhị phân
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # Biên dịch mô hình
    adam_optimizer = optimizers.Adam(learning_rate=1e-5)
    model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',  # Theo dõi loss trên tập validation
    patience=5,          # Dừng sau 5 epoch nếu không cải thiện
    restore_best_weights=True
    )

    log_name = datetime.now().strftime("%Hh%Mp__%d-%m-%Y")+"_MLP.csv"
    csv_logger = CSVLogger(log_name , append=True)
    model.fit(X_train_scaled, y_train, epochs=30, validation_split=0.1, batch_size=16, verbose = 1, callbacks=[early_stopping, csv_logger])
    print("Huấn luyện hoàn tất!")

    # Bước 3: Đánh giá mô hình
    print("[3/3] Đánh giá hiệu suất trên tập kiểm thử...")
    # Chuyển đổi xác suất đầu ra thành nhãn 0 hoặc 1
    y_pred_proba = model.predict(X_test_scaled)
    y_pred = (y_pred_proba > 0.5).astype("int32")
    
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['An toàn (Benign)', 'Độc hại (Malicious)'])

    print("\n================== KẾT QUẢ ĐÁNH GIÁ (MLP) ==================")
    print(f"✅ Độ chính xác (Accuracy): {accuracy:.4f} ({accuracy*100:.2f}%)")
    print("\n📊 Báo cáo Phân loại (Classification Report):")
    print(report)
    print("\n🔀 Ma trận Nhầm lẫn (Confusion Matrix):")
    print(confusion_matrix(y_test, y_pred))

if __name__ == '__main__':
    train_mlp_model(INPUT_FILE)

--- Huấn luyện mô hình Multi-Layer Perceptron (MLP) ---
[1/3] Đã chuẩn bị xong dữ liệu.
[2/3] Xây dựng và huấn luyện mô hình MLP...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,201 (12.50 KB)

 Trainable params: 3,201 (12.50 KB)

...

Epoch 1/30


I0000 00:00:1751476524.476523  108397 service.cc:152] XLA service 0x7f68780049f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751476524.476574  108397 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-07-03 00:15:24.495692: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751476524.618428  108397 cuda_dnn.cc:529] Loaded cuDNN version 90300


 56/405 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7555 - loss: 0.6329

I0000 00:00:1751476525.199199  108397 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


405/405 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7423 - loss: 0.6380 - val_accuracy: 0.9471 - val_loss: 0.5960
Epoch 2/30
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8292 - loss: 0.6035 - val_accuracy: 0.9485 - val_loss: 0.5586
Epoch 3/30
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8873 - loss: 0.5647 - val_accuracy: 0.9597 - val_loss: 0.5242
Epoch 4/30
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9216 - loss: 0.5351 - val_accuracy: 0.9694 - val_loss: 0.4909
Epoch 5/30
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9407 - loss: 0.5016 - val_accuracy: 0.9750 - val_loss: 0.4582
Epoch 6/30
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9506 - loss: 0.4713 - val_accuracy: 0.9805 - val_loss: 0.4289
Epoch 7/30
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9520 - loss: 0.4427 - val_accuracy: 0.9833 - val_loss: 0.4005
Epoch 8/30
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9599 - loss: 0.4159 - val_accuracy: 0.9861 - val_

In [10]:
# train_widedeep
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# --- Cấu hình ---
INPUT_FILE = 'npm_shuffled.csv'

def create_widedeep_model(input_dim):
    """Tạo mô hình Wide & Deep sử dụng Functional API."""
    # Input layer chung
    inputs = tf.keras.layers.Input(shape=(input_dim,))
    
    # 1. Nhánh "Deep" (là một MLP)
    deep_path = tf.keras.layers.Dense(128, activation='relu')(inputs)
    deep_path = tf.keras.layers.BatchNormalization()(deep_path)
    deep_path = tf.keras.layers.Dropout(0.3)(deep_path)
    deep_path = tf.keras.layers.Dense(64, activation='relu')(deep_path)
    deep_path = tf.keras.layers.BatchNormalization()(deep_path)
    deep_path = tf.keras.layers.Dropout(0.2)(deep_path)
    
    # 2. Nhánh "Wide" (chỉ là đầu vào đi thẳng)
    # Trong trường hợp này, nhánh Wide chính là đầu vào ban đầu.
    wide_path = inputs
    
    # 3. Kết hợp hai nhánh
    merged_path = tf.keras.layers.concatenate([wide_path, deep_path])
    
    # Lớp đầu ra
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(merged_path)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

def train_widedeep(input_file):
    print(f"--- Huấn luyện mô hình Wide & Deep ---")

    # Bước 1: Tải và chuẩn bị dữ liệu
    df = pd.read_csv(input_file)
    X = df.drop(columns=['label', 'package_name'])
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    print(f"[1/3] Đã chuẩn bị xong dữ liệu.")

    # Bước 2: Xây dựng và Huấn luyện
    model = create_widedeep_model(X_train_scaled.shape[1])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    print("[2/3] Bắt đầu huấn luyện mô hình Wide & Deep...")
    model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)
    print("Huấn luyện hoàn tất!")

    # Bước 3: Đánh giá
    print("[3/3] Đánh giá hiệu suất...")
    y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
    
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['An toàn (Benign)', 'Độc hại (Malicious)'])
    print("\n================== KẾT QUẢ ĐÁNH GIÁ (Wide & Deep) ==================")
    print(f"✅ Độ chính xác (Accuracy): {accuracy:.4f}")
    print("\n📊 Báo cáo Phân loại:")
    print(report)
    print("\n🔀 Ma trận Nhầm lẫn (Confusion Matrix):")
    print(confusion_matrix(y_test, y_pred))
    
if __name__ == '__main__':
    train_widedeep(INPUT_FILE)

--- Huấn luyện mô hình Wide & Deep ---
[1/3] Đã chuẩn bị xong dữ liệu.


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_27 (Dense)    │ (None, 128)       │      2,176 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_27[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 128)       │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_28 (Dense)    │ (None, 64)        │      8,256 │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_28[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 64)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 80)        │          0 │ input_layer_6[0]… │
│ (Concatenate)       │                   │            │ dropout_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_29 (Dense)    │ (None, 1)         │         81 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,281 (44.07 KB)

 Trainable params: 10,897 (42.57 KB)

 Non-trainable params: 384 (1.50 KB)

[2/3] Bắt đầu huấn luyện mô hình Wide & Deep...
Epoch 1/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9052 - loss: 0.2331 - val_accuracy: 0.9889 - val_loss: 0.1294
Epoch 2/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9858 - loss: 0.0754 - val_accuracy: 0.9889 - val_loss: 0.0716
Epoch 3/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9882 - loss: 0.0623 - val_accuracy: 0.9889 - val_loss: 0.0624
Epoch 4/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9858 - loss: 0.0724 - val_accuracy: 0.9889 - val_loss: 0.0572
Epoch 5/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9861 - loss: 0.0692 - val_accuracy: 0.9889 - val_loss: 0.0568
Epoch 6/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9887 - loss: 0.0595 - val_accuracy: 0.9889 - val_loss: 0.0572
Epoch 7/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9883 - loss: 0.0608 - val_accuracy: 0.9889 - val_loss: 0.0574
Epoch 8/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - ac

In [9]:
# train_resnet
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# --- Cấu hình ---
INPUT_FILE = 'npm_shuffled.csv'

def residual_block(x, units, dropout_rate=0.3):
    """Một khối Residual Block cơ bản."""
    # Nhánh chính
    fx = tf.keras.layers.Dense(units, activation='relu')(x)
    fx = tf.keras.layers.BatchNormalization()(fx)
    fx = tf.keras.layers.Dropout(dropout_rate)(fx)
    fx = tf.keras.layers.Dense(units)(fx) # Lớp thứ hai không có activation
    
    # Kết nối tắt (Skip Connection)
    # Nếu số unit khác nhau, cần một lớp Dense để thay đổi kích thước
    if x.shape[-1] != units:
        x = tf.keras.layers.Dense(units)(x)
        
    # Cộng nhánh chính và kết nối tắt
    output = tf.keras.layers.add([x, fx])
    output = tf.keras.layers.Activation('relu')(output)
    output = tf.keras.layers.BatchNormalization()(output)
    return output

def create_resnet_model(input_dim):
    """Tạo mô hình ResNet sử dụng Functional API."""
    inputs = tf.keras.layers.Input(shape=(input_dim,))
    
    # Một lớp Dense ban đầu để xử lý đầu vào
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    
    # Chồng các khối residual
    x = residual_block(x, units=64)
    x = residual_block(x, units=64)
    
    # Lớp đầu ra
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

def train_resnet(input_file):
    print(f"--- Huấn luyện mô hình Residual Network (ResNet) ---")

    # Bước 1: Tải và chuẩn bị dữ liệu
    df = pd.read_csv(input_file)
    X = df.drop(columns=['label', 'package_name'])
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    print(f"[1/3] Đã chuẩn bị xong dữ liệu.")

    # Bước 2: Xây dựng và Huấn luyện mô hình
    model = create_resnet_model(X_train_scaled.shape[1])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    print("[2/3] Bắt đầu huấn luyện mô hình ResNet...")
    model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)
    print("Huấn luyện hoàn tất!")

    # Bước 3: Đánh giá
    print("[3/3] Đánh giá hiệu suất...")
    y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
    
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['An toàn (Benign)', 'Độc hại (Malicious)'])
    print("\n================== KẾT QUẢ ĐÁNH GIÁ (ResNet) ==================")
    print(f"✅ Độ chính xác (Accuracy): {accuracy:.4f}")
    print("\n📊 Báo cáo Phân loại:")
    print(report)
    print("\n🔀 Ma trận Nhầm lẫn (Confusion Matrix):")
    print(confusion_matrix(y_test, y_pred))

if __name__ == '__main__':
    train_resnet(INPUT_FILE)

--- Huấn luyện mô hình Residual Network (ResNet) ---
[1/3] Đã chuẩn bị xong dữ liệu.


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 64)        │      1,088 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 64)        │      4,160 │ dense_21[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_22[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 64)        │      4,160 │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 64)        │          0 │ dense_21[0][0],   │
│                     │                   │            │ dense_23[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64)        │          0 │ add_2[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ activation_2[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 64)        │      4,160 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_24[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, 64)        │      4,160 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 64)        │          0 │ batch_normalizat… │
│                     │                   │            │ dense_25[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 64)        │          0 │ add_3[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ activation_3[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_26 (Dense)    │ (None, 1)         │         65 │ batch_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 18,817 (73.50 KB)

 Trainable params: 18,305 (71.50 KB)

 Non-trainable params: 512 (2.00 KB)

[2/3] Bắt đầu huấn luyện mô hình ResNet...
Epoch 1/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8716 - loss: 0.2894 - val_accuracy: 0.9889 - val_loss: 0.1349
Epoch 2/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9861 - loss: 0.0778 - val_accuracy: 0.9889 - val_loss: 0.0599
Epoch 3/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9862 - loss: 0.0699 - val_accuracy: 0.9889 - val_loss: 0.0570
Epoch 4/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9865 - loss: 0.0681 - val_accuracy: 0.9889 - val_loss: 0.0582
Epoch 5/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9888 - loss: 0.0657 - val_accuracy: 0.9889 - val_loss: 0.0566
Epoch 6/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9871 - loss: 0.0673 - val_accuracy: 0.9889 - val_loss: 0.0571
Epoch 7/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9848 - loss: 0.0795 - val_accuracy: 0.9889 - val_loss: 0.0556
Epoch 8/20
203/203 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accur

In [8]:
# train_autoencoder
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# --- Cấu hình ---
INPUT_FILE = 'npm_shuffled.csv'
ENCODING_DIM = 8 # Kích thước của lớp bottleneck (lớp mã hóa)

def train_autoencoder_model(input_file):
    print(f"--- Huấn luyện mô hình Autoencoder cho Phát hiện Bất thường ---")

    # Bước 1: Tải và chuẩn bị dữ liệu
    try:
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{input_file}'.")
        return

    X = df.drop(columns=['label', 'package_name'])
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # QUAN TRỌNG: Chỉ huấn luyện Autoencoder trên dữ liệu AN TOÀN (benign)
    X_train_benign = X_train[y_train == 0]
    print(f"[1/4] Đã chuẩn bị dữ liệu. Sẽ huấn luyện trên {len(X_train_benign)} mẫu an toàn.")

    # Bước 2: Xây dựng mô hình Autoencoder
    input_dim = X.shape[1]
    autoencoder = tf.keras.models.Sequential([
        # Encoder
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(ENCODING_DIM, activation='relu'), # Lớp bottleneck

        # Decoder
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(input_dim, activation='sigmoid') # Tái tạo lại đầu vào
    ])

    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    autoencoder.summary()

    # Bước 3: Huấn luyện mô hình
    print("[2/4] Bắt đầu huấn luyện Autoencoder...")
    autoencoder.fit(
        X_train_benign, X_train_benign, # Đầu vào và đầu ra đều là dữ liệu an toàn
        epochs=50,
        batch_size=32,
        shuffle=True,
        validation_split=0.1,
        verbose=1
    )
    print("Huấn luyện hoàn tất!")

    # Bước 4: Đánh giá bằng lỗi tái tạo
    print("[3/4] Tính toán lỗi tái tạo trên tập kiểm thử...")
    reconstructions = autoencoder.predict(X_test)
    mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)
    
    # Xác định ngưỡng để phân loại bất thường
    # Chúng ta giả định rằng 95% các mẫu an toàn sẽ có lỗi dưới ngưỡng này.
    benign_test_errors = mse[y_test == 0]
    threshold = np.quantile(benign_test_errors, 0.95)
    print(f"Ngưỡng lỗi tái tạo được xác định là: {threshold:.4f}")

    # Dự đoán: Bất cứ mẫu nào có lỗi > ngưỡng đều là độc hại (1)
    print("[4/4] Đánh giá hiệu suất...")
    y_pred = (mse > threshold).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['An toàn (Benign)', 'Độc hại (Malicious)'])

    print("\n================== KẾT QUẢ ĐÁNH GIÁ (Autoencoder) ==================")
    print(f"✅ Độ chính xác (Accuracy): {accuracy:.4f} ({accuracy*100:.2f}%)")
    print("\n📊 Báo cáo Phân loại (Classification Report):")
    print(report)
    print("\n🔀 Ma trận Nhầm lẫn (Confusion Matrix):")
    print(confusion_matrix(y_test, y_pred))

if __name__ == '__main__':
    train_autoencoder_model(INPUT_FILE)

--- Huấn luyện mô hình Autoencoder cho Phát hiện Bất thường ---
[1/4] Đã chuẩn bị dữ liệu. Sẽ huấn luyện trên 2399 mẫu an toàn.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 16)             │           528 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,424 (9.47 KB)

 Trainable params: 2,424 (9.47 KB)

 Non-trainable params: 0 (0.00 B)

[2/4] Bắt đầu huấn luyện Autoencoder...
Epoch 1/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 1.4387 - val_loss: 1.7445
Epoch 2/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9736 - val_loss: 1.6205
Epoch 3/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3234 - val_loss: 1.5822
Epoch 4/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8863 - val_loss: 1.5634
Epoch 5/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8719 - val_loss: 1.5477
Epoch 6/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9737 - val_loss: 1.5186
Epoch 7/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9428 - val_loss: 1.4992
Epoch 8/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1193 - val_loss: 1.4948
Epoch 9/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9591 - val_loss: 1.4922
Epoch 10/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8708 - val_loss: 1.4909
Epoch 11/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2212 - val_loss: 1.4892
Epoch 12/50
68/68 ━━━━━━━━━━━━━━━